In [15]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments


In [2]:
MODEL_NAME = "distilbert-base-uncased"
NUM_LABELS = 2

# 1. Load dataset (use your dataset)
# Here we use the 'ag_news' small sample as a placeholder. Replace with your dataset.
raw = load_dataset("ag_news", split="train[:2000]")
raw = raw.train_test_split(test_size=0.1)
train_ds = raw["train"]
eval_ds = raw["test"]



s:\VSCode\myenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Adnan Habib\.cache\huggingface\hub\datasets--ag_news. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better perform

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)


train_ds = train_ds.map(preprocess, batched=True)
eval_ds = eval_ds.map(preprocess, batched=True)

train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
eval_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# 3. Load base model (frozen base weights are default; Trainer will only update LoRA params after wrapping)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS)

# 4. Create LoRA config and wrap the model
lora_config = LoraConfig(
    r=8,                # rank
    lora_alpha=32,      # scaling
    target_modules=["q", "v"],  # module names to apply LoRA to (common for attention layers)
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# model = get_peft_model(model, lora_config)
# for name, module in model.named_modules():
#     print(name)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_lin", "v_lin"],   
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


 
print_trainable_params(model)

Map: 100%|██████████| 200/200 [00:00<00:00, 5432.02 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925
Trainable params: 739586 / 67694596 (1.093%)


In [18]:
training_args = TrainingArguments(
    output_dir="lora-distilbert",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    logging_steps=50,
    learning_rate=5e-5,
    fp16=False,
    save_steps=1000,  # manually save every N steps
    do_eval=True      # evaluate at logging_steps intervals
)


In [20]:
def compute_metrics(eval_pred):
    import numpy as np
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = (preds == labels).mean()
    return {"accuracy": acc}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

C:\Users\Adnan Habib\AppData\Local\Temp\ipykernel_6196\1193539808.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()


s:\VSCode\myenv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


IndexError: Target 2 is out of bounds.